<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>LightGBM - Clase 5 - V2.0</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [1]:

#Importing libraries

import pandas as pd
import numpy as np


In [2]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1000)

# Muestra las primeras filas del DataFrame
print(data_full.head())
data_full.info()
print(df_pid_validos.head())

#  Si tomo el archivo desde el buket de la maquina virtual del laboratorio
#
# Ruta del archivo ventas comprimido
# file_path = 'data/sell-in.txt.gz'

# Abre el archivo comprimido
#with gzip.open(file_path, 'rt') as f:
#    # Lee el archivo CSV usando pandas, especificando que el separador es un tabulador
#    data_full = pd.read_csv(f, sep='\t')



   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_

In [3]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

La lista product_ids_validos tiene 780 registros.


In [4]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

# Ver como esta el data frame
print(data.head())
data.info()

El DataFrame filtrado tiene 2293481 registros.
Todos los product_id en el DataFrame filtrado son válidos.
Número de registros por product_id en el DataFrame filtrado:
product_id
20111    7973
20122    7950
20120    7537
20326    7397
20132    7199
         ... 
21267      67
21252      67
21276      64
20886      63
20953      62
Name: count, Length: 780, dtype: int64
   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
<class 'pan

In [5]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [7]:

# Hacer una copia del DataFrame original para evitar SettingWithCopyWarning
data = data.copy()

# Transformaciones iniciales
data['periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')
data['year'] = data['periodo'].dt.year
data['month'] = data['periodo'].dt.month

In [8]:
# Agregar ventas por producto y período
data_agg = data.groupby(['product_id', 'year', 'month']).agg({
    'tn': 'sum',
    'plan_precios_cuidados': 'max',  # Asumimos que si algún registro tiene 1, el producto estaba en el plan
    'cust_request_qty': 'sum'
}).reset_index()

In [9]:
# Identificar los productos más vendidos
def filter_products(data, product_id_min, product_id_max):
    return data[(data['product_id'] >= product_id_min) & (data['product_id'] <= product_id_max)]

In [24]:
# Filtrar para los productos seleccionados
product_id_min = 20000  # Ejemplo: límite inferior
product_id_max = 21277  # Ejemplo: límite superior
filtered_data = filter_products(data_agg, product_id_min, product_id_max)

In [25]:
# Crear variables dummy para product_id
product_dummies = pd.get_dummies(filtered_data['product_id'], prefix='prod')
filtered_data = pd.concat([filtered_data, product_dummies], axis=1)

In [26]:
# Recalcular características históricas
filtered_data = filtered_data.sort_values(['product_id', 'year', 'month'])
filtered_data['hist_sales'] = filtered_data.groupby('product_id')['tn'].shift(1).fillna(0)
filtered_data['cum_sales'] = filtered_data.groupby('product_id')['tn'].cumsum() - filtered_data['tn']

In [27]:
# Preparación para el entrenamiento
features = ['year', 'month', 'plan_precios_cuidados', 'cust_request_qty', 'hist_sales', 'cum_sales'] + list(product_dummies.columns)
X = filtered_data[features]
y = filtered_data['tn']

# División en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Entrenar el modelo LightGBM
model = lgb.LGBMRegressor(objective='regression', metric='rmse', num_leaves=31, learning_rate=0.05, feature_fraction=0.9, bagging_fraction=0.8, bagging_freq=5, verbose=0)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, feature_fraction=0.9,
              learning_rate=0.05, metric='rmse', objective='regression',
              verbose=0)

In [33]:
# Preparación de datos para la predicción (febrero 2020)
pred_data = pd.DataFrame()

# Calcular el promedio de clientes por producto en los últimos 6 períodos de venta
def avg_customers_last_6_periods(product_id):
    product_data = data[data['product_id'] == product_id].copy()
    product_data['periodo'] = pd.to_datetime(product_data['periodo'])
    product_data = product_data.sort_values('periodo')
    
    # Identificar los últimos 6 períodos únicos con ventas
    last_6_periods = product_data['periodo'].unique()[-6:]
    
    # Filtrar los datos para esos 6 períodos
    filtered_product_data = product_data[product_data['periodo'].isin(last_6_periods)]
    
    # Calcular el número promedio de clientes
    customers_per_period = filtered_product_data.groupby('periodo')['customer_id'].nunique()
    return customers_per_period.mean()

# Calculamos el promedio de clientes para cada producto
avg_customers = {product: avg_customers_last_6_periods(product) for product in filtered_data['product_id'].unique()}

for product in filtered_data['product_id'].unique():
    product_data = filtered_data[filtered_data['product_id'] == product].sort_values(['year', 'month'])
    
    # Calculamos la tendencia de ventas (crecimiento promedio en los últimos 6 meses)
    last_6_months = product_data.iloc[-6:]
    if len(last_6_months) >= 2:
        sales_trend = (last_6_months['tn'].iloc[-1] / last_6_months['tn'].iloc[0]) ** (1/len(last_6_months)) - 1
    else:
        sales_trend = 0
    
    new_row = {
        'product_id': product,
        'year': 2020,
        'month': 2,
        'plan_precios_cuidados': product_data['plan_precios_cuidados'].iloc[-1],
        'cust_request_qty': product_data['cust_request_qty'].mean(),
        'hist_sales': product_data['tn'].iloc[-1],
        'cum_sales': product_data['tn'].sum(),
        'avg_customers': avg_customers[product],
        'sales_trend': sales_trend
    }
    pred_data = pd.concat([pred_data, pd.DataFrame([new_row])], ignore_index=True)

# Crear dummies para los productos en los datos de predicción
pred_product_dummies = pd.get_dummies(pred_data['product_id'], prefix='prod')
pred_data = pd.concat([pred_data, pred_product_dummies], axis=1)

# Asegurar que todas las columnas dummy estén presentes y en el mismo orden que en el conjunto de entrenamiento
for col in product_dummies.columns:
    if col not in pred_data.columns:
        pred_data[col] = 0

# Verificar y asegurar que `X_pred` tenga las mismas características que `features`
X_pred = pred_data[features]

# Realizar predicciones
base_predictions = model.predict(X_pred)

# Ajustar las predicciones
adjusted_predictions = base_predictions * pred_data['avg_customers']
adjusted_predictions *= (1 + 0.5 * pred_data['sales_trend']) / 125

# Asignar las predicciones ajustadas de vuelta a `pred_data`
pred_data['tn'] = adjusted_predictions.round(4)  # Ajustar a 4 decimales

# Guardar las predicciones en un archivo CSV
output = pred_data[['product_id', 'tn']]
output.to_csv('LIGHTGBM_pred_feb_2020.v3.0.4.csv', index=False)
print("Predicciones guardadas en 'LIGHTGBM_pred_feb_2020.csv'")

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Predicciones guardadas en 'LIGHTGBM_pred_feb_2020.csv'


<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 